In [8]:
import os  # 운영 체제와 상호작용하기 위한 모듈
import shutil  # 파일 및 디렉토리 작업을 위한 모듈
import random  # 랜덤 작업을 위한 모듈
import scipy  # 과학 및 기술 계산을 위한 모듈
import numpy as np  # 수치 연산을 위한 Python 라이브러리
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리
from PIL import Image  # 이미지 처리 및 파일 입출력을 위한 라이브러리
from keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터 생성기 유틸리티
from keras.models import Sequential, Model  # Keras의 시퀀셜 및 함수형 API 모델
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, GlobalAveragePooling2D, Add, Activation  # 신경망의 레이어 구성 요소
from keras.optimizers import Adam, SGD  # 최적화 알고리즘
from keras.callbacks import EarlyStopping, ModelCheckpoint  # 학습 중 콜백 함수

def generators(train_dir, val_dir):
    """학습 및 검증 데이터를 생성하는 함수"""
    # 학습 데이터 생성기 설정
    train_datagen = ImageDataGenerator(rescale=1/255,  # 이미지의 픽셀 값을 0-1 범위로 정규화
                                       rotation_range=180,  # 최대 180도 회전
                                       width_shift_range=0.2,  # 가로 방향 이동
                                       height_shift_range=0.2,  # 세로 방향 이동
                                       shear_range=0.2,  # 시어 변환
                                       zoom_range=0.2,  # 줌 인/줌 아웃
                                       horizontal_flip=True,  # 가로 방향 뒤집기
                                       vertical_flip=True,  # 세로 방향 뒤집기
                                       fill_mode='nearest')  # 변환 중 생기는 빈 공간을 주변의 유사한 픽셀 값으로 채우기

    # 검증 데이터 생성기 설정
    val_datagen = ImageDataGenerator(rescale=1/255)  # 이미지의 픽셀 값을 0-1 범위로 정규화

    # 학습 데이터 생성
    train_generator = train_datagen.flow_from_directory(train_dir,  # 학습 데이터가 위치한 디렉토리
                                                        target_size=(224, 224),  # 입력 이미지 크기
                                                        batch_size=16,  # 배치 크기
                                                        class_mode='categorical',  # 다중 클래스 분류
                                                        shuffle=True)  # 데이터를 섞어서 배치 생성

    # 검증 데이터 생성
    val_generator = val_datagen.flow_from_directory(val_dir,  # 검증 데이터가 위치한 디렉토리
                                                    target_size=(224, 224),  # 입력 이미지 크기
                                                    batch_size=16,  # 배치 크기
                                                    class_mode='categorical')  # 다중 클래스 분류

    return train_generator, val_generator  # 학습 및 검증 데이터 생성기 반환

# 학습 및 검증 데이터 디렉토리 설정
train_dir = 'your_path'  # 학습 데이터 디렉토리 경로
val_dir = 'your_path'  # 검증 데이터 디렉토리 경로

# 데이터 생성기 생성
train_generator, val_generator = generators(train_dir, val_dir)

def conv1(x):
    """첫 번째 Convolution 블록을 정의하는 함수"""
    x = ZeroPadding2D(padding=(1, 1))(x)  # 패딩 추가
    x = Conv2D(64, (3, 3), strides=(1, 1))(x)  # 첫 번째 Conv2D 레이어
    x = Conv2D(64, (3, 3), strides=(1, 1))(x)  # 두 번째 Conv2D 레이어
    x = Conv2D(64, (3, 3), strides=(1, 1))(x)  # 세 번째 Conv2D 레이어
    x = BatchNormalization()(x)  # 배치 정규화 레이어
    x = Activation('relu')(x)  # 활성화 함수 적용
    x = ZeroPadding2D(padding=(1, 1))(x)  # 추가 패딩
    return x
    
def conv2(x):
    """두 번째 Convolution 블록을 정의하는 함수"""
    x = MaxPooling2D((3, 3), 2)(x)  # 맥스풀링 레이어
    
    shortcut = x  # 스킵 연결용 단축 경로 저장

    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)  # 스킵 연결 Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            shortcut = BatchNormalization()(shortcut)  # 스킵 연결 배치 정규화
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
    return x

def conv3(x):
    """세 번째 Convolution 블록을 정의하는 함수"""
    shortcut = x  # 스킵 연결용 단축 경로 저장

    for i in range(4):
        if(i == 0):
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)  # 스킵 연결 Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            shortcut = BatchNormalization()(shortcut)  # 스킵 연결 배치 정규화
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
    return x

def conv4(x):
    """네 번째 Convolution 블록을 정의하는 함수"""
    shortcut = x  # 스킵 연결용 단축 경로 저장

    for i in range(6):
        if(i == 0):
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)  # 스킵 연결 Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            shortcut = BatchNormalization()(shortcut)  # 스킵 연결 배치 정규화
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
    return x

def conv5(x):
    """다섯 번째 Convolution 블록을 정의하는 함수"""
    shortcut = x  # 스킵 연결용 단축 경로 저장

    for i in range(3):
        if(i == 0):
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)  # 스킵 연결 Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            shortcut = BatchNormalization()(shortcut)  # 스킵 연결 배치 정규화
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Activation('relu')(x)  # 활성화 함수 적용
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)  # Conv2D 레이어
            x = BatchNormalization()(x)  # 배치 정규화 레이어
            x = Add()([x, shortcut])  # 스킵 연결 합성
            x = Activation('relu')(x)  # 활성화 함수 적용
            shortcut = x  # 현재 출력값을 다음 반복을 위한 단축 경로로 설정
    return x

def custom():
    """ResNet 스타일의 커스텀 모델을 정의하는 함수"""
    input_tensor = Input(shape=(224,224,3))  # 입력 텐서 정의
    
    x = conv1(input_tensor)  # 첫 번째 Convolution 블록 적용
    x = conv2(x)  # 두 번째 Convolution 블록 적용
    x = conv3(x)  # 세 번째 Convolution 블록 적용
    x = conv4(x)  # 네 번째 Convolution 블록 적용
    x = conv5(x)  # 다섯 번째 Convolution 블록 적용
    x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링 레이어 적용
    output = Dense(100, activation='softmax')(x)  # 출력 레이어 (Softmax 활성화 함수 사용)
    
    model = Model(input_tensor, output)  # 모델 정의
    model.compile(optimizer=SGD(learning_rate=.01, momentum=.9, decay=.001), loss='categorical_crossentropy', metrics=['accuracy'])  # 모델 컴파일
    model.summary()  # 모델 요약 출력

    return model

# 커스텀 모델 생성
model = custom()

# 모델 저장 경로 및 콜백 설정
model_path = 'your_path'  # 최적의 모델을 저장할 파일 경로
CP = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)  # 최적의 모델을 저장하는 콜백
ES = EarlyStopping(monitor='val_loss', patience=10)  # 조기 종료를 위한 콜백, 10 에포크 동안 개선이 없으면 학습 중단

# 모델 학습
history = model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[CP, ES])  # 학습 수행

# 학습 결과 시각화
plt.plot(history.history['loss'])  # 학습 손실 그래프
plt.plot(history.history['accuracy'])  # 학습 정확도 그래프
plt.plot(history.history['val_loss'])  # 검증 손실 그래프
plt.plot(history.history['val_accuracy'])  # 검증 정확도 그래프
plt.title('Custom Model')  # 그래프 제목
plt.ylabel('loss')  # y축 라벨
plt.xlabel('epoch')  # x축 라벨
plt.legend(['loss', 'accuracy', 'val_loss', 'val_accuracy'], loc='upper right')  # 범례
plt.show()  # 그래프 출력

# 최적의 검증 정확도와 손실 출력
print('Best Val Acc:', max(history.history['val_accuracy']))  # 최고 검증 정확도 출력
print('Best Val Loss:', min(history.history['val_loss']))  # 최저 검증 손실 출력